In [1]:
import pygame 
import neat
import time
import os
import random
pygame.font.init()



pygame 2.3.0 (SDL 2.24.2, Python 3.9.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
WIN_WIDTH=600
WIN_HEIGHT=800
GEN=0



BIRD_IMGS=[pygame.transform.scale2x(pygame.image.load("bird1.png")),pygame.transform.scale2x(pygame.image.load("bird2.png")),pygame.transform.scale2x(pygame.image.load("bird3.png"))]
PIPE_IMG=pygame.transform.scale2x(pygame.image.load("pipe.png"))
BASE_IMG=pygame.transform.scale2x(pygame.image.load("base.png"))
BG_IMG=pygame.transform.scale2x(pygame.image.load("bg.png"))

STAT_FONT =pygame.font.SysFont("comicsans",50)

class Bird:
    """
    Bird class representing the flappy bird
    """
    MAX_ROTATION = 25
    IMGS = BIRD_IMGS
    ROT_VEL = 20
    ANIMATION_TIME = 5

    def __init__(self, x, y):
        """
        Initialize the object
        :param x: starting x pos (int)
        :param y: starting y pos (int)
        :return: None
        """
        self.x = x
        self.y = y
        self.tilt = 0  # degrees to tilt
        self.tick_count = 0
        self.vel = 0
        self.height = self.y
        self.img_count = 0
        self.img = self.IMGS[0]

    def jump(self):
        """
        make the bird jump
        :return: None
        """
        self.vel = -10.5
        self.tick_count = 0
        self.height = self.y

    def move(self):
        """
        make the bird move
        :return: None
        """
        self.tick_count += 1

        # for downward acceleration
        d = self.vel*(self.tick_count) + 0.5*(3)*(self.tick_count)**2  # calculate displacement

        # terminal velocity
        if d >= 16:
            d = (d/abs(d)) * 16

        if d < 0:
            d -= 2

        self.y = self.y + d

        if d < 0 or self.y < self.height + 50:  # tilt up
            if self.tilt < self.MAX_ROTATION:
                self.tilt = self.MAX_ROTATION
        else:  # tilt down
            if self.tilt > -90:
                self.tilt -= self.ROT_VEL

    def draw(self, win):
        """
        draw the bird
        :param win: pygame window or surface
        :return: None
        """
        self.img_count += 1

        # For animation of bird, loop through three images
        if self.img_count <= self.ANIMATION_TIME:
            self.img = self.IMGS[0]
        elif self.img_count <= self.ANIMATION_TIME*2:
            self.img = self.IMGS[1]
        elif self.img_count <= self.ANIMATION_TIME*3:
            self.img = self.IMGS[2]
        elif self.img_count <= self.ANIMATION_TIME*4:
            self.img = self.IMGS[1]
        elif self.img_count == self.ANIMATION_TIME*4 + 1:
            self.img = self.IMGS[0]
            self.img_count = 0

        # so when bird is nose diving it isn't flapping
        if self.tilt <= -80:
            self.img = self.IMGS[1]
            self.img_count = self.ANIMATION_TIME*2


        # tilt the bird
        rotated_image = pygame.transform.rotate(self.img, self.tilt)
        new_rect = rotated_image.get_rect(center = self.img.get_rect(topleft = (self.x,self.y)).center)

        win.blit(rotated_image, new_rect.topleft)

    def get_mask(self):
        """
        gets the mask for the current image of the bird
        :return: None
        """
        return pygame.mask.from_surface(self.img)
    
    
    
    
class Pipe:
    GAP=200
    VEL=5
    
    def __init__(self,x):
        self.x=x
        self.height=0
        
        
        self.top=0
        self.bottom=0
        self.PIPE_TOP=pygame.transform.flip(PIPE_IMG,False,True)
        self.PIPE_BOTTOM=PIPE_IMG
        
        self.passed=False
        self.set_height()
        
    def set_height(self):
        self.height = random.randrange(50,450)
        self.top=self.height-self.PIPE_TOP.get_height()
        self.bottom=self.height+self.GAP
        
    def move(self):
        self.x -= self.VEL
        
    def draw(self,win):
        win.blit(self.PIPE_TOP,(self.x,self.top))
        win.blit(self.PIPE_BOTTOM,(self.x,self.bottom))
        
        
    def collide(self,bird):
        bird_mask=bird.get_mask()
        top_mask=pygame.display.from_surface(self.PIPE_TOP)
        bottom.mask=pygame.display.from_surface(self.PIPE_BOTTOM)
        top_offset=(self.x-bird.x,self.top-round(bird.y))
        bottom_offset=(self.x-bird.x,self.bottom-round(bird.y))
        b_point=bird_mask.overlap(bottom_mask,bottom_offset)
        t_point=bird_mask.overlap(top_mask,top_offset) 
        if t_point or b_point:
             return True
            
        return False
    
class Base:
    VEL=5
    WIDTH=BASE_IMG.get_width()
    IMG=BASE_IMG

    def __init__(self,y):
        self.y=y
        self.x1=0                                 
        self.x2=self.WIDTH
                                         
    def move(self):
      
        self.x1 -= self.VEL
        self.x2 -= self.VEL
        if self.x1 + self.WIDTH < 0:
            self.x1 = self.x2 + self.WIDTH

        if self.x2 + self.WIDTH < 0:
            self.x2 = self.x1 + self.WIDTH
    
    def draw(self,win):
        win.blit(self.IMG,(self.x1,self.y))                                 
    
        win.blit(self.IMG,(self.x2,self.y))                                  
                                         
                                         
def draw_window(win,birds,pipes,base,score,gen):
    win.blit(BG_IMG,(0,0))
    for pipe in pipes:
        pipe.draw(win)
        
    text=STAT_FONT.render("Score: "+ str(score),1,(255,255,255))
    win.blit((10,10))
    
    text=STAT_FONT.render("Gen"+ str(gen),1,(255,255,255))
    win.blit(text,(10,10))
    base.draw(win)    
    for bird in birds:
        
        bird.draw(win)
    pygame.display.update()
    
def main(genomes,config):
    global GEN
    GEN+=1
    
    net=[]
    ge=[]
    birds=[]
    
    for _,g in genomes:
        net=neat.nn.FeedForwardNetwork.create(g,config)
        nets=append(net)
        birds.append(Bird(230,350))
        g.fitness=0
        ge.append(g)
        
        
    
    
    base=Base(730)
    pipes=[Pipe(700)]
    win=pygame.display.set_mode((WIN_WIDTH,WIN_HEIGHT))
    clock=pygame.time.Clock()
    score=0

    run=True
    while run:
        for event in pygame.event.get():
            if event.type==pygame.QUIT:
                run=False
                pygame.quit()
                quit()
        pipe_ind=0 
        
    
        if len(birds)>0:
            if len(pipes)>1 and bird[0].x>pipes[0].x + PIPE_TOP.get_width():
                pipe_ind=1
        else:
            run=False
            break;
            
        for x,bird in enumerate(birds):
            bird.move()
            ge[x].fitness+=0.1
            
            output=nets[x].activate((bird.y,abs(bird.y-pipes[pipe_ind].height),abs(bird.y-pipes[pipes_ind].bottom)))
            
            if output[0]>0.5:
                bird.jump()
            
            
        
        
        add_pipe=False
        rem=[]
        for pipe in pipes:
            for x,bird in enumerate(birds):
                
                if pipe.collide(bird):
                    ge[x].fitness-=1
                    birds.pop(x)
                    nets.pop(x)
                    ge.pop(x)
                    
                
                
            
                if not pipe.passed and pipe.x<bird.x:
                    pipe.passed=True
                    add_pipe=True
                    
            if pipe.x+pipe.PIPE_TOP.get_width()<0:
                rem.append(pipe)      
            pipe.move()    
                
            if add_pipe:
                score+=1
                for g in ge:
                    g.fitness+=5
                pipes.append(Pipe(700))
            for r in rem:
                pipes.remove(r)
            
            
            for x,bird in enumerate(birds):
                
                if bird.y+bird.img.get_heigth()>= 730 or bird.y<0:
                    birds.pop(x)
                    nets.pop(x)
                    ge.pop(x)
                    
            
                
            
            
        base.move()
        draw_window(win,birds,pipes,base,score)
                
  
    
    


def run():
    congig=neat.confi.Config(neat.DefaultGenome,DefaultReproduction,neat.DefaultSpeciesSet,neat.DefaultStagnation,config_path)
    p=neat.Population(config)
    p.add_reporter(neat.StdOutReporter(True))
    stats=neat.StatisticsReporter()
    p.add_reporter(stats)
    winner=p.run(main,50)

if __name__=="__main__":
    local_dir=os.path.dirname(__file__)
    config_path = os.path.join(local_dir,"config-feedforward.txt")
    run(config_path)

            
        
    
    
        
                
        
    


NameError: name '__file__' is not defined

In [ ]:

'''[NEAT]
fitness_criterion     = max
fitness_threshold     = 100
pop_size              = 20
reset_on_extinction   = False

[DefaultGenome]
# node activation options
activation_default      = tanh
activation_mutate_rate  = 0.0
activation_options      = tanh

# node aggregation options
aggregation_default     = sum
aggregation_mutate_rate = 0.0
aggregation_options     = sum

# node bias options
bias_init_mean          = 0.0
bias_init_stdev         = 1.0
bias_max_value          = 30.0
bias_min_value          = -30.0
bias_mutate_power       = 0.5
bias_mutate_rate        = 0.7
bias_replace_rate       = 0.1

# genome compatibility options
compatibility_disjoint_coefficient = 1.0
compatibility_weight_coefficient   = 0.5

# connection add/remove rates
conn_add_prob           = 0.5
conn_delete_prob        = 0.5

# connection enable options
enabled_default         = True
enabled_mutate_rate     = 0.01

feed_forward            = True
initial_connection      = full

# node add/remove rates
node_add_prob           = 0.2
node_delete_prob        = 0.2

# network parameters
num_hidden              = 0
num_inputs              = 3
num_outputs             = 1

# node response options
response_init_mean      = 1.0
response_init_stdev     = 0.0
response_max_value      = 30.0
response_min_value      = -30.0
response_mutate_power   = 0.0
response_mutate_rate    = 0.0
response_replace_rate   = 0.0

# connection weight options
weight_init_mean        = 0.0
weight_init_stdev       = 1.0
weight_max_value        = 30
weight_min_value        = -30
weight_mutate_power     = 0.5
weight_mutate_rate      = 0.8
weight_replace_rate     = 0.1

[DefaultSpeciesSet]
compatibility_threshold = 3.0

[DefaultStagnation]
species_fitness_func = max
max_stagnation       = 20
species_elitism      = 2

[DefaultReproduction]
elitism            = 2
survival_threshold = 0.2'''


